# Data Loading, Storage and File Formats
## DAT540 Introduction to Data Science
## University of Stavanger
### L08
#### Antorweep Chakravorty (antorweep.chakravorty@uis.no)

In [3]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from lxml import objectify
from yr.libyr import Yr
import sqlite3
import sqlalchemy as sqla


- File inputs and outputs with pandas
- Reading text and other formats from disk
- loading data from disk, databases and networks (web APIs)
- pandas data loading (read) instance methods

<img src='./images/loadfiles.png' width='450'>

- HDF5 package is a interface to the HDF5 binary data format. 
  - store huge amounts of numerical data, and easily manipulate that data from NumPy
  - can slice into multi-terabyte datasets stored on disk, as if they were real NumPy arrays
  - thousands of datasets can be stored in a single file, categorized and tagged
- msgpack is an efficient binary serialization format. It lets you exchange data among multiple languages 
- **pickle** module implements binary protocols for serializing and de-serializing a Python object structure
- sas files stored as either XPORT or SAS7BDAT format files.
- **feather** provides binary columnar serialization for data frames
  - It is designed to make reading and writing data frames efficient 
  - allows sharing data across different data analysis languages

- pandas read_* instance methods load and converts the data into a DataFrame
- Optional arguments for these functions may fall into few categories:
  - *Indexing* can treat one or more columns as indices from the returned DataFrame, and to determine column names 
  - *Type interface and data conversion* includes the user-defined value conversions and custom list of missing value markers
  - *Datatime parsing* including combining capability, combining date and time information spread over multiple columns into a single column
  - *Iterating* support for iterating over chunks of very large files
  - *Unclean data issues* skipping rows or a footer, comments or other aspects nuances

- reading **CSVs** (read_csv)
- Read CSV (comma-separated) file into DataFrame

In [1]:
# let us check the structure of file ex1.csv from filesystem
!head ./data/ex1.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [4]:
# using pd.read_csv to read this file directly will set the delimiter to comma (,)
# If the delimiter is different we have the pass the delimiter argument with the delimiting value
ex1 = pd.read_csv('./data/ex1.csv')
ex1

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [5]:
# However, since this file doesnot have any header names we can ignore it by passing the argument header=None
ex1 = pd.read_csv('./data/ex1.csv', header=None)
ex1

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# If we rather want to manually rename the columns we can use the names=[] argument (without the header argument)
ex1 = pd.read_csv('./data/ex1.csv', names=['zero', 'one', 'two', 'three', 'four'])
ex1

,zero,one,two,three,four
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# In order to set a column as the index for the rows, we can use the argument index_col.
# For multi/hierarchical index we pass a list of columns
# The columns can be indices or labels
ex1 = pd.read_csv('./data/ex1.csv', names=['zero', 'one', 'two', 'three', 'four'], index_col=['four'])
ex1

,zero,one,two,three
four,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
! head ./data/ex2.csv

aaa -0.264436 -1.026059 -0.619500
bbb  0.927272  0.3029   -0.032399
ccc -0.264273 -0.386914 -0.217601 
dd  -0.871858 -0.348382  1.100491


- How would you load this data set
- Is there a header
- What would be the delimiter

In [11]:
# We would use regular expression as delimiter
# \s+ indicates one or more whitespaces
ex2 = pd.read_csv('data/ex2.csv', header=None, delimiter='\s+')
ex2

,0,1,2,3
0,aaa,-0.264436,-1.026059,-0.619500
1,bbb,0.927272,0.302900,-0.032399
2,ccc,-0.264273,-0.386914,-0.217601
3,dd,-0.871858,-0.348382,1.100491


- Cases where a file might have rows that are not relevent to the overall dataset, can be skipped using argument *skiprow* list  of row indices

In [12]:
!cat ./data/ex3.csv

# Example 03
# Headers as follows
A,B,C,Message
# The content of the table is represented bellow
1,2,3,hello
4,5,6,world
7,8,9,foo


In [13]:
ex3 = pd.read_csv('./data/ex3.csv', skiprows=[0,1,3])
ex3

,A,B,C,Message
0,1,2,3,hello
1,4,5,6,world
2,7,8,9,foo


In [14]:
!head ./data/titanic.csv

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,,S
5,6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",,2.0,3,1,349909,21.075,,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,,S


- How will you load the titanic.csv file into a dataframe
- Set hierarchical index on columns: Sex, Age

In [15]:
titanic = pd.read_csv('./data/titanic.csv', delimiter=',', index_col=['Sex', 'Age'])
titanic.head(3)

Unnamed: 0  PassengerId  Survived  Pclass  \
Sex    Age                                               
male   22.0           0            1         0       3   
female 38.0           1            2         1       1   
       26.0           2            3         1       3   

                                                          Name  SibSp  Parch  \
Sex    Age                                                                     
male   22.0                            Braund, Mr. Owen Harris      1      0   
female 38.0  Cumings, Mrs. John Bradley (Florence Briggs Th...      1      0   
       26.0                             Heikkinen, Miss. Laina      0      0   

                       Ticket     Fare Cabin Embarked  
Sex    Age                                             
male   22.0         A/5 21171   7.2500   NaN        S  
female 38.0          PC 17599  71.2833   C85        C  
       26.0  STON/O2. 3101282   7.9250   NaN        S

- Some of the frequently used options in pandas.read_csv

<img src='./images/read_csv.png' width=800, height=400>

- *Reading Text Files in Pieces*
- Used when processing very large files or figuring out the right set of arguments
- Also useful to load a section of the file, visualize top rows and figure out its structure
- The *nrows* argument can be supplied with an integer value to read only that many rows and not the whole file
- To read a file in pieces, the argument *chunksize* can be set as a number of rows 
  - A *TextParser* object is returned that allows iteration over the parts
  - A *get_chunk* method is also available to TextParser that enables pieces of arbitrary size to be read

In [16]:
titanic = pd.read_csv('./data/titanic.csv', nrows=5)
titanic.shape

(5, 13)

In [17]:
# We can also read the data in chunks
titanic = pd.read_csv('./data/titanic.csv', chunksize=100, delimiter=',')
titanic

In [18]:
# Here we want to count the freq. of ages
age = pd.Series([], dtype=float)
for data in titanic:
  age = age.add(data['Age'].value_counts(), fill_value=0)

In [19]:
print('size:', age.size)
age.head()

size: 56


0.83    1.0
2.00    3.0
3.00    1.0
4.00    2.0
5.00    1.0
dtype: float64

- *Writing Data to Text Format*
- Using a Series or DataFrame *to_csv(filename)* instance method, the data struct, can be written to disk.
- The default delimiter is a comma, however, it can be overridden using the *delimiter* argument
- *na_rep* argument can be used to denote missing values by some sentinel value rather than empty string
- By default, the row and column labels are written to disk. This can be overwritten using attributes *index=False* and/or *header=False*
- using *columns=[]* argument, columns can be filtered and ordered while writing to disk

In [20]:
titanic = pd.read_csv('./data/titanic.csv', delimiter=',')

- Write the titanic object to disk with filename titanicmod.csv
- Suppress row labels/indices
- use comma as delimiter
- only select these columns in the given order [PassengerId, Age, Sex, Survived]

In [21]:
titanic.to_csv('./data/titanicmod.csv', columns=['PassengerId', 'Age', 'Sex', 'Survived'], index=False)

In [22]:
!head ./data/titanicmod.csv

PassengerId,Age,Sex,Survived
1,22.0,male,0
2,38.0,female,1
3,26.0,female,1
4,35.0,female,1
5,35.0,male,0
6,,male,0
7,54.0,male,0
8,2.0,,0
9,27.0,female,1


- *Working with Delimited Formats*
- How will you convert the following file to a DataFrame

In [23]:
!cat ./data/ex4.rtf

{\rtf1\ansi\ansicpg1252\cocoartf1671
{\fonttbl\f0\fswiss\fcharset0 Helvetica;}
{\colortbl;\red255\green255\blue255;}
{\*\expandedcolortbl;;}
\paperw11900\paperh16840\margl1440\margr1440\vieww10800\viewh8400\viewkind0
\pard\tx566\tx1133\tx1700\tx2267\tx2834\tx3401\tx3968\tx4535\tx5102\tx5669\tx6236\tx6803\pardirnatural\partightenfactor0

\f0\fs24 \cf0 #values in both header and data are delimited with a comma\
#Header\
ID,C1,C2,C3,Value\
#Data\
1,1,2,3,x\
2,4,5,6,y\
3,7,8,9,z}

In [24]:
# load/read the file as a table
ex4 = pd.read_table('./data/ex4.rtf', nrows=10, header=None)
ex4

,0
0,{\rtf1\ansi\ansicpg1252\cocoartf1671
1,{\fonttbl\f0\fswiss\fcharset0 Helvetica;}
2,{\colortbl;\red255\green255\blue255;}
3,{\*\expandedcolortbl;;}
4,\paperw11900\paperh16840\margl1440\margr1440\v...
5,\pard\tx566\tx1133\tx1700\tx2267\tx2834\tx3401...
6,\f0\fs24 \cf0 #values in both header and data ...
7,#Header\
8,"ID,C1,C2,C3,Value\"
9,#Data\


In [25]:
# load/read the file as a table and skip rows that are not required
ex4 = pd.read_table('./data/ex4.rtf', header=None, skiprows=list(range(0,9)) + [10])
ex4

,0
0,"ID,C1,C2,C3,Value\"
1,"1,1,2,3,x\"
2,"2,4,5,6,y\"
3,"3,7,8,9,z}"


In [26]:
# Now let us load it as a csv
ex4 = pd.read_csv('./data/ex4.rtf', header=None, skiprows=list(range(0,9)) + [10])
ex4

,0,1,2,3,4
0,ID,C1,C2,C3,Value\
1,1,1,2,3,x\
2,2,4,5,6,y\
3,3,7,8,9,z}


In [27]:
# remove the special chars in col 4
ex4.replace(to_replace=r'\\*}*', value='', regex=True, inplace=True)
ex4

,0,1,2,3,4
0,ID,C1,C2,C3,Value
1,1,1,2,3,x
2,2,4,5,6,y
3,3,7,8,9,z


In [28]:
# Now let us load it as a csv with the first row as header
ex4 = pd.read_csv('./data/ex4.rtf', header=0, skiprows=list(range(0,9)) + [10])
# remove the special chars in col 4
ex4.replace(to_replace=r'\\*}*', value='', regex=True, inplace=True)
ex4

# How can we do replace the special char in the header without using replace on columns attribute

,ID,C1,C2,C3,Value\
0,1,1,2,3,x
1,2,4,5,6,y
2,3,7,8,9,z


In [29]:
# Now let us load it as a csv with header as None
ex4 = pd.read_csv('./data/ex4.rtf', header=None, skiprows=list(range(0,9)) + [10])
# remove the special chars in col 4
ex4.replace(to_replace=r'\\*}*', value='', regex=True, inplace=True)

# Set the proper row and col labels
ex4.columns = ex4.iloc[0]
ex4.drop(0, inplace=True)
ex4.set_index('ID', drop=True, inplace=True)
ex4

,C1,C2,C3,Value
ID,,,,
1,1,2,3,x
2,4,5,6,y
3,7,8,9,z


- *JSON (JavaScript Object Notation) Data*
- a standard format for sending data over HTTP requests 
```javascript
{'name': 'Wes',
'places_lived': ['US', 'NO', 'UK', 'DE'],
'pet': null,
'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['D', 'Zuko']},
             {'name': 'Katie', 'age': 38,
              'pets': ['Sixes', 'Stache', 'Cisco']}]}```

- The module *json* is the default library for operating on JSON file formats
- To convert a JSON string to Python, the *json.loads* method is used
- To convert a Python object back to JSON, *json.dump* method is used
- *pandas.read_json* can automatically convert JSON datasets into a Series or a DataFrame
  - it assumes that each object in the JSON array is a row in a table
- pandas can export data to JSON format using *pandas.to_json* method on a Series or a DataFrame

In [30]:
!head ./data/jsonex.json

{
    "quiz": {
        "sport": {
            "q1": {
                "question": "Which one is correct team name in NBA?",
                "options": [
                    "New York Bulls",
                    "Los Angeles Kings",
                    "Golden State Warriros",
                    "Huston Rocket"


In [35]:
# Reading, using pandas
pd.read_json('./data/jsonex.json')

,quiz
maths,"{'q1': {'question': '5 + 7 = ?', 'options': ['..."
sport,{'q1': {'question': 'Which one is correct team...


- However, *read_json* only produces two rows in this case
  - This is because it assumes that each object in the JSON array is a row in a table
- Inorder to create a DataFrame with all elements as a cell we have to flatten the json object
- We use *json_normalize* from pandas.io.json for flattening json on pandas DataFrame

In [36]:
json_normalize(pd.read_json('./data/jsonex.json').loc['maths'])

/var/folders/b5/qtlwb6j979d58ssnfbkgyckc0000gn/T/ipykernel_27937/3806011293.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(pd.read_json('./data/jsonex.json').loc['maths'])


,q1.question,q1.options,q1.answer,q2.question,q2.options,q2.answer
0,5 + 7 = ?,"[10, 11, 12, 13]",12,12 - 8 = ?,"[1, 2, 3, 4]",4


- *XML and HTML: Web Scraping*
- Multiple libraries are available for processing HTML & XML formats
  - lxml, BeautifulSoup, html5lib
- pandas has a build-in function *read_html* that uses lxml and Beautiful Soup to automatically parse tables out of HTML as DataFrames
- read_html takes both local and remote html files as input

In [42]:
tables = pd.read_html('https://www.fi.muni.cz/~hales/html/tables.html')
print('Number of retrived tables:', len(tables))

Number of retrived tables: 53


- XML is another structured data format that is more generic than compared to HTML
- We load plant_catalog.xml as a DataFrame
- In order to load a xml format, we have to first import class *objectify* from module lxml
- We use to instance method *parse* from objectify to parse the xml file to lxml object
- next we iterate through the lxml object and create the relevant dataframe by choosing the required elements

In [43]:
! head ./data/plant_catalog.xml

<?xml version="1.0" encoding="UTF-8"?>
<CATALOG>
  <PLANT>
    <COMMON>Bloodroot</COMMON>
    <BOTANICAL>Sanguinaria canadensis</BOTANICAL>
    <ZONE>4</ZONE>
    <LIGHT>Mostly Shady</LIGHT>
    <PRICE>$2.44</PRICE>
    <AVAILABILITY>031599</AVAILABILITY>
  </PLANT>


In [44]:
obj = objectify.parse(open('./data/plant_catalog.xml'))

In [45]:
# We store all the leaf nodes
data = []
# We choose not to select some properties of the each plant
skip_tag = ['AVAILABILITY']
# We get a generator yielding each root element CATALOG
root = obj.getroot()
# We go through each root element and get their child nodes

for element in root:  
  for child in element.getchildren():   
    leafnodes = {}
    for leaf in child.getchildren():        
      if leaf.tag not in skip_tag:
        leafnodes[leaf.tag] = leaf.text
    data.append(leafnodes)

df = pd.DataFrame(data)
df.head(2)

,COMMON,BOTANICAL,ZONE,LIGHT,PRICE
0,Bloodroot,Sanguinaria canadensis,4,Mostly Shady,$2.44
1,Columbine,Aquilegia canadensis,3,Mostly Shady,$9.37


- *Binary Data Formats*
- Serialization is a mechanism to store data in binary format
- Python, provides a built-in module *pickle* for serialization
- pandas objects have a *to_pickle* instance method that writes data to disk in pickle format
- pickled data can be read from disk using the *pandas.read_pickle* method

In [46]:
df.to_pickle('./data/plant_catalog')

- pandas also provides opportunities to read an Excel file
- The instance method *read_excel* can be used to read a excel file
  - along with the file path
  - a string or integer value can be provides as a second argument depicting the the Sheet Name or index
  - a list of string or integers can also be provided as a second argument to load multiple sheets
  - *None* is provided as the second argument to load all sheets
  - When a single value is provided, a DataFrame is returned
  - With None or list as the second argument, a *Dict* of DataFrames are returned having the keys representing the sheets
  - Inorder to write as excel, we first create an *pd.ExcelWriter* and then add DataFrames to the excel writer as sheets using instance method *to_excel(writer, SheetName)*

```python
writer = pd.ExcelWriter('./data/output.xlsx')
df1.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()
```